In [16]:
import glob
import os
import shutil

from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Observations

import numpy as np
import pandas as pd

%matplotlib inline

In [17]:
postflash_proposal_list = [16570,16982] #[13078,13560,14006,14372,14535,14984,15573,15718,16398,16570,16982]
working_directory = '/Users/cmartlin/Desktop/postflash_2022/'
database_filename = working_directory + '/Jan_2023_all_postflash_flc.pkl'

In [12]:
postflash_proposals = Observations.query_criteria(proposal_id=postflash_proposal_list)

science_list = postflash_proposals

Observations.download_products(science_list['obsid'], mrp_only=False, download_dir='./science',
                                productSubGroupDescription=['FLC', 'RAW'])

Local Path,Status,Message,URL
str55,str8,object,object
./science/mastDownload/HST/ielu07raq/ielu07raq_raw.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu07raq/ielu07raq_flc.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu06q5q/ielu06q5q_raw.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu06q5q/ielu06q5q_flc.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu07qxq/ielu07qxq_raw.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu07qxq/ielu07qxq_flc.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu07rgq/ielu07rgq_raw.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu07rgq/ielu07rgq_flc.fits,COMPLETE,None,None
./science/mastDownload/HST/ielu07r3q/ielu07r3q_raw.fits,COMPLETE,None,None


In [18]:
files_fits = glob.glob(working_directory + '/mastDownload/HST/*/*flc.fits')
print(len(files_fits))
test = files_fits[:]

0


In [ ]:
file_name = []
prop_id = []
paths = []
rootname = []
obsdate = []
obstime = []
shutter = []
flash_lvl = []
flash_dur = []

flash_cur = []
subarray = []
exp_time = []

for f in test:
    print(f)
    try:
        data = fits.open(f)
        rootname_val = data[0].header['ROOTNAME']
        filename_val = data[0].header['FILENAME']
        propid_val = data[0].header['PROPOSID']
        date_val = data[0].header['DATE-OBS']
        time_val = data[0].header['TIME-OBS']
        exptime_val = data[0].header['EXPTIME']
        subarray_val = data[0].header['SUBARRAY']
        flash_dur_val = data[0].header['FLASHDUR']
        flash_cur_val = data[0].header['FLASHCUR']
        flash_lvl_val = data[0].header['FLASHLVL']
        shutter_val = data[0].header['SHUTRPOS']
        print(rootname_val)

        file_name.append(filename_val)
        prop_id.append(propid_val)
        paths.append(f)
        rootname.append(rootname_val)
        obsdate.append(date_val)
        obstime.append(time_val)
        shutter.append(shutter_val)
        flash_lvl.append(flash_lvl_val)
        flash_dur.append(flash_dur_val)
        flash_cur.append(flash_cur_val)
        subarray.append(subarray_val)
        exp_time.append(exptime_val)

        data.close()
    except: 
        print('File is corrupt and could not open.')


In [ ]:
dict = {'filename':file_name,'propid': prop_id, 'path': paths, 'rootname': rootname, 
        'obsdate': obsdate, 'obstime': obstime, 'shutter': shutter, 'flash_lvl': flash_lvl,
        'flash_dur': flash_dur, 'flash_cur': flash_cur, 'subarray': subarray,
        'exp_time': exp_time}

postflash_data = pd.DataFrame(dict) 

postflash_data = postflash_data.set_index('rootname')

postflash_data.to_pickle(database_filename)

In [ ]:
postflash_data

In [ ]:
postflash_data = pd.read_pickle(database_filename)
subarray_pf = postflash_data.loc[(postflash_data['subarray'] == True)] 
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
sub_paths = subarray_pf.path.tolist()
all_paths = postflash_data.path.tolist()
print (all_paths)


In [ ]:
means = []
sig_means = []
medians = []
sig_medians = []
stds = []
sig_stds = []

for f in all_paths:
    path = f
    print(path)
    data = fits.getdata(path)
    data_clip = stats.sigma_clip(data, 3)
    
    mean = np.mean(data)
    sig_mean = np.mean(data_clip)
    median = np.median(data)
    sig_median = np.median(data_clip)
    std = np.std(data)
    sig_std = np.std(data_clip)
    
    
    means.append(mean)
    sig_means.append(sig_mean)
    medians.append(median)
    sig_medians.append(sig_median)
    stds.append(std)
    sig_stds.append(sig_std)

In [ ]:
dict = {'path': all_paths, 'mean': means, 'sig_mean': sig_means,'median': medians, 'sig_median': sig_medians, 'std': stds, 'sig_std': sig_stds}
path_and_stats_sub = pd.DataFrame(dict)

all_stats = pd.merge(left=postflash_data, right=path_and_stats_sub, left_on='path', right_on='path').set_index(postflash_data.index)

In [ ]:
all_stats['norm_by_dur'] = (all_stats['sig_mean']/all_stats['flash_dur'])
all_stats.to_pickle(updated_filename)
all_stats = pd.read_pickle(updated_filename)
print (all_stats)

In [ ]:
all_stats['datetime'] = pd.to_datetime(all_stats['obsdate'] + ' ' + all_stats['obstime'])
all_stats.to_pickle(updated_filename)
